This aim of this notebook is to figure out how to get data from spotify. 

Resources : https://medium.com/@rafaelnduarte/how-to-retrieve-data-from-spotify-110c859ab304


A faire aussi : créer un compte spotify https://developer.spotify.com/ et créer une app, puis rajouter dans les authoristions le redirect_uri utilisé

In [1]:
import spotipy 
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
import spotipy.util as util

In [2]:
# setting up authorization
cid = "d219b776c1574adb99e52a6a3592fb6e" 
secret = "d81ae5a6d57645d2a3274cce038b688b"
# saving the info you're going to need
username = 'emmamilliot'
scope = 'user-library-read' #check the documentation
authorization_url = 'https://accounts.spotify.com/authorize'
token_url = 'https://accounts.spotify.com/api/token'
redirect_uri ='http://localhost:8808/callback/'

token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri=redirect_uri)
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
# retrieving you access token
auth = SpotifyClientCredentials(
client_id=cid,
client_secret=secret)
# save your token
token = auth.get_access_token()
spotify = spotipy.Spotify(auth=token)
# check if everything is in order
print(token)
print(spotify)

{'access_token': 'BQCbSf1e5PRlbgIAb3uW0u6jbO9RRXBlDFBrewknPtG5amYvoJizkQlTT8p9o1iDsszbv55jNKkalDO1huo', 'token_type': 'Bearer', 'expires_in': 3600, 'expires_at': 1647015110}


<ipython-input-3-4d0cc25606e1>:6: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = auth.get_access_token()


In [4]:
from spotipy.client import SpotifyException

In [5]:
with open('genres.txt') as f:
    genres = f.readlines()
genres = [genres[k].replace('\n', '') for k in range(len(genres))]

In [6]:
# create empty lists where the results are going to be stored
artist_name = []
artist_id = []
artists_id = []
artists_names = []
track_name = []
popularity = []
track_id = []
years = []
genres_list = []

for year in [2017, 2022] : 
    print(year)
    qu = '+year:{}'.format(year)
    for i in range(0,1000,50):
        print(i)
        count_exceptions = 0
        for genre in genres[0:100]:
            # print(genre)
            try : 
                query = 'genre:"{}"'.format(genre) + qu
                track_results = sp.search(q=query, type='track', limit=50, offset=i)
                # if len(track_results['tracks']['items']) == 0:
                #     empty_list += 1
                for i, t in enumerate(track_results['tracks']['items']):
                    artist_name.append(t['artists'][0]['name'])
                    artist_id.append(t['artists'][0]['id'])
                    artists_id.append([t['artists'][k]['id'] for k in range(len(t['artists']))])
                    artists_names.append([t['artists'][k]['name'] for k in range(len(t['artists']))])
                    track_name.append(t['name'])
                    track_id.append(t['id'])
                    popularity.append(t['popularity'])
                    years.append(year)
                    genres_list.append(genre)
            except SpotifyException:
                count_exceptions += 1
                if count_exceptions < 10 : 
                    pass
                else:
                    raise
            except : 
                raise
            # if empty_list > 10 : 
            #     break

2017
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
2022
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [7]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,
                        'track_name':track_name,
                        'track_id':track_id,
                        'popularity':popularity, 
                        'artist_id' : artist_id, 
                        'artists_id' : artists_id,
                        'artists_names' : artists_names, 
                        'year' : years,
                        'genre' : genres_list})
print(df_tracks.shape)
# check if everything is ok
df_tracks.head()

(200000, 9)


,artist_name,track_name,track_id,popularity,artist_id,artists_id,artists_names,year,genre
0,Jaymes Young,Infinity,1SOClUWhOi8vHZYMz3GluK,96,6QrQ7OrISRYIfS5mtacaw2,[6QrQ7OrISRYIfS5mtacaw2],[Jaymes Young],2017,pop
1,Ruth B.,Dandelions,2eAvDnpXP5W0cVtiI0PUxV,93,2WzaAvm2bBCf4pEhyuDgCY,[2WzaAvm2bBCf4pEhyuDgCY],[Ruth B.],2017,pop
2,Chase Atlantic,Swim,3M0lSi5WW79CXQamgSBIjx,89,7cYEt1pqMgXJdq00hAwVpT,[7cYEt1pqMgXJdq00hAwVpT],[Chase Atlantic],2017,pop
3,Måneskin,Beggin',3Wrjm47oTz2sjIgck11l5e,92,0lAWpj5szCSwM4rUMHYmrr,[0lAWpj5szCSwM4rUMHYmrr],[Måneskin],2017,pop
4,Offset,Ric Flair Drip (with Metro Boomin),7sO5G9EABYOXQKNPNiE9NR,85,4DdkRBBYG6Yk9Ka8tdJ9BW,"[4DdkRBBYG6Yk9Ka8tdJ9BW, 0iEtIxbK0KxaSlF7G42ZOp]","[Offset, Metro Boomin]",2017,pop


In [8]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

6535

In [9]:
# dropping duplicates

df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [10]:
df_tracks.to_csv('tracks_2017_2020.csv')

In [11]:
# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    print(i)
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
Number of tracks where no audio features were available: 0


In [12]:
# saving the features in a data frame
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns') 
print("Shape of the dataset:", df_audio_features.shape) 
df_audio_features.head()

Shape of the dataset: (7272, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.671,0.673,11,-6.941,0,0.0444,0.1480,0.000053,0.3040,0.4940,121.963,audio_features,1SOClUWhOi8vHZYMz3GluK,spotify:track:1SOClUWhOi8vHZYMz3GluK,https://api.spotify.com/v1/tracks/1SOClUWhOi8v...,https://api.spotify.com/v1/audio-analysis/1SOC...,237720,4
1,0.609,0.692,1,-2.958,1,0.0259,0.0157,0.000000,0.0864,0.4540,116.959,audio_features,2eAvDnpXP5W0cVtiI0PUxV,spotify:track:2eAvDnpXP5W0cVtiI0PUxV,https://api.spotify.com/v1/tracks/2eAvDnpXP5W0...,https://api.spotify.com/v1/audio-analysis/2eAv...,233720,3
2,0.666,0.542,8,-6.429,1,0.0392,0.2700,0.000000,0.0765,0.0771,120.134,audio_features,3M0lSi5WW79CXQamgSBIjx,spotify:track:3M0lSi5WW79CXQamgSBIjx,https://api.spotify.com/v1/tracks/3M0lSi5WW79C...,https://api.spotify.com/v1/audio-analysis/3M0l...,228787,4
3,0.714,0.800,11,-4.808,0,0.0504,0.1270,0.000000,0.3590,0.5890,134.002,audio_features,3Wrjm47oTz2sjIgck11l5e,spotify:track:3Wrjm47oTz2sjIgck11l5e,https://api.spotify.com/v1/tracks/3Wrjm47oTz2s...,https://api.spotify.com/v1/audio-analysis/3Wrj...,211560,4
4,0.880,0.428,9,-8.280,1,0.2060,0.1490,0.000051,0.1140,0.3330,100.007,audio_features,7sO5G9EABYOXQKNPNiE9NR,spotify:track:7sO5G9EABYOXQKNPNiE9NR,https://api.spotify.com/v1/tracks/7sO5G9EABYOX...,https://api.spotify.com/v1/audio-analysis/7sO5...,172800,4


In [13]:
df_audio_features.to_csv('raw_data\\audio_feaures_2017_and_2020.csv')

In [14]:

df_tracks.describe()

,popularity,year
count,7272.000000,7272.000000
mean,44.576320,2019.096397
std,24.367509,2.467376
min,0.000000,2017.000000
25%,36.000000,2017.000000
50%,52.000000,2017.000000
75%,61.000000,2022.000000
max,96.000000,2022.000000
